# Table of Contents
| Section                      | Subsections                |
|-------------------------------|----------------------------|
| [Exploratory Data Analysis](#exploratory-data-analysis) | [Data Cleaning](#data-cleaning), [Data Visualization](#data-visualization) |
| [Feature Engineering](#feature-engineering) | [Creating New Features](#creating-new-features), [Transformations](#transformations) |

## Introduction

Critical minerals are crucial for technologies that drive global energy, security, and economic systems. This exploratory analysis will focus on Rare Earth Elements (REEs), highlighting their unique and essential properties. Here are some key points to consider:

- The concentration of REE production primarily in China ([figure 1](https://unctad.org/publication/digital-economy-report-2024)) exposes global supply chains to geopolitical and logistical vulnerabilities, underscoring the need for diversified supply sources [[INN](https://investingnews.com/rare-earths-forecast/)].

- According to the IEA’s Global Critical Minerals Outlook, the major concern in the rare earths sector is not the demand-supply gap but the high geographical concentration of mining and refining activities. This concentration significantly heightens the risk of supply disruptions in the market [[INN](https://investingnews.com/rare-earths-forecast/)].

- Strategically, governments are increasingly viewing REEs as critical minerals, prompting investments across the exploration, extraction, and processing stages of the rare earth supply chain [[MN](https://www.miningnewswire.com/three-key-trends-that-could-impact-rare-earths-in-2024/)]. 

## Exploratory Data Analysis

_Deposit has dimensions of approximately 400 m north-south by 100-250 m east-west._
importante para justificar el tamanho del buffer y de la resolution de rasters


Zonal statistics in buffers (prospective points in CM dataset)

## Feature Engineering